In [1]:
import numpy as np

In [2]:
def gf2_rank(rows):
    """
    Find rank of a matrix over GF2.

    The rows of the matrix are given as nonnegative integers, thought
    of as bit-strings.

    This function modifies the input list. Use gf2_rank(rows.copy())
    instead of gf2_rank(rows) to avoid modifying rows.
    """
    rank = 0
    while len(rows):
        pivot_row = rows[-1]
        rows = np.delete(rows, -1, 0)
        if np.count_nonzero(pivot_row):
            rank += 1
            lsb = pivot_row & -pivot_row
            for index, row in enumerate(rows):
                if np.count_nonzero(row & lsb):
                    rows[index] = row ^ pivot_row
    return rank
def rref(H):
    r = 0
    n = len(H[0])
    num_checks = len(H)
    k = n - num_checks
    for c in range(0, num_checks):
        if (H[r, c] == 0):
            # need to swap
            for r2 in range(r+1, num_checks):
                if (H[r2, c] == 1):
                    temp = H[r, :].copy()
                    H[r, :] = H[r2, :]
                    H[r2, :] = temp
                    break
                    
        # if (H[r, c] == 0):
        #     print('H is singular')

        # cancel out other rows
        for r2 in range(r+1, num_checks):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] ^ H[r, :]

        # # back substitution
        for r2 in range(0, r):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] ^ H[r, :]

        r += 1
    return H

In [11]:
def repetition_code(n):
    # generates the parity check matrix for a repitition code with parameters [n, 1, n]

    H = np.zeros((n-1, n), dtype=int)
    for i in range(n-1):
        H[i][i] = 1
        H[i][i+1] = 1

    return H
H = repetition_code(15)
print(H)

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]


In [67]:
H = repetition_code(15)

vs = np.array([], dtype=int)
cs = np.array([], dtype=int)

local_perc = 0.5
# desired_rate = 0.25

for i in range(int(H.shape[0] * local_perc)):
    nz_rows = np.nonzero(np.any(H, axis=1))[0]
    c_ind = np.random.choice(nz_rows)

    for v_ind in np.where(H[c_ind])[0]:
        vs = np.append(vs, v_ind)
    cs = np.append(cs, [c_ind, c_ind])
    H[c_ind] = np.zeros(n)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            vs = np.delete(vs, v_ind)
            cs =np.delete(cs, c_ind)

for i in range(int(3)):
    H = np.delete(H, i, axis=0)    
print(H)

[[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]]


In [68]:
n = H.shape[1]
k = n - H.shape[0]
print(n, k)

print(f"Rank: {gf2_rank(H)}")
rref_H = rref(H.copy())
G = np.hstack([np.eye(k, dtype=int), rref_H[:,n-k:n].T])
print(G)

15 4
Rank: 11
[[1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 1 0 0 1 0 1 0 0 1 0 0 0 1 1]
 [0 0 1 0 1 1 0 0 0 1 1 0 0 1 0]
 [0 0 0 1 1 1 0 0 0 0 0 1 1 0 0]]


In [69]:
for j in range(0, 2**n):
    num = bin(j)[2:].zfill(n)
    w = np.array([int(b) for b in num])
    # encoded_w = w@G % 2

    # print(w)
    if (not np.any(H@w % 2)):
        print(w)
    # if (np.count_nonzero(encoded_w) < d):
        # d = np.count_nonzero(encoded_w)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 1 1 1 1 1 0 1 1 1]
[0 0 0 1 1 0 0 0 0 0 0 1 0 0 0]
[0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 1 1 1 1 1 1 0 1 1 1]
[0 1 1 1 1 0 0 0 0 0 0 1 0 0 0]
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 1 1 1 1 1 1 0 1 1 1]
[1 0 0 1 1 0 0 0 0 0 0 1 0 0 0]
[1 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 0 1 1 1 1 1 1 0 1 1 1]
[1 1 1 1 1 0 0 0 0 0 0 1 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [55]:
H = np.array([[1,1],[1,-1]]) * 1/np.sqrt(2)
S = np.array([[1,0],[0,1j]])
Sd = np.array([[1,0],[0,-1j]])

X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
I = np.eye(2)

zero = np.array([1,0])
one = np.array([0,1])

plus = np.array([1,1]) * 1/np.sqrt(2)
minus = np.array([1,-1]) * 1/np.sqrt(2)

r = np.array([1,1j]) * 1/np.sqrt(2)
l = np.array([1,-1j]) * 1/np.sqrt(2)

bell = np.array([1,0,0,1]) * 1/np.sqrt(2)
ghz = np.array([1,0,0,0,0,0,0,1]) * 1/np.sqrt(2)

In [38]:
zz = np.kron(zero, one)
proj = np.outer(zz, zz)
prob = (bell @ np.kron(H,I).conj().T) @ (proj.conj().T @ proj) @ (np.kron(H,I) @ bell)
print(prob)

0.2499999999999999


In [62]:
basis1 = H@Sd
basis2 = H

zz = np.kron(zero, zero)
proj = np.outer(zz, zz)
prob = (bell @ np.kron(basis1,basis2).conj().T) @ (proj.conj().T @ proj) @ (np.kron(basis1,basis2) @ bell)
print(f"Pr(00) = {round(abs(prob),2)}")
zz = np.kron(zero, one)
proj = np.outer(zz, zz)
prob = (bell @ np.kron(basis1,basis2).conj().T) @ (proj.conj().T @ proj) @ (np.kron(basis1,basis2) @ bell)
print(f"Pr(01) = {round(abs(prob),2)}")
zz = np.kron(one, zero)
proj = np.outer(zz, zz)
prob = (bell @ np.kron(basis1,basis2).conj().T) @ (proj.conj().T @ proj) @ (np.kron(basis1,basis2) @ bell)
print(f"Pr(10) = {round(abs(prob),2)}")
zz = np.kron(one, one)
proj = np.outer(zz, zz)
prob = (bell @ np.kron(basis1,basis2).conj().T) @ (proj.conj().T @ proj) @ (np.kron(basis1,basis2) @ bell)
print(f"Pr(11) = {round(abs(prob),2)}")

Pr(00) = 0.25
Pr(01) = 0.25
Pr(10) = 0.25
Pr(11) = 0.25


In [39]:
proj @ (np.kron(H,I) @ bell) / np.sqrt(prob)

array([0., 1., 0., 0.])

In [47]:
zzz = np.kron(one, np.kron(one, zero))
proj = np.outer(zzz, zzz)
(ghz @ np.kron(H,np.kron(H,H)).conj().T) @ (proj.conj().T @ proj) @ (np.kron(H,np.kron(H,H)) @ ghz)

0.24999999999999983

In [96]:
b1 = H
b2 = I
b3 = I

for i in range(2):
    for j in range(2):
        for k in range(2):
            oc1 = one if i else zero
            oc2 = one if j else zero
            oc3 = one if k else zero

            zzz = np.kron(oc1, np.kron(oc2, oc3))
            proj = np.outer(zzz, zzz)
            prob = (ghz @ np.kron(b1,np.kron(b2,b3)).conj().T) @ (proj.conj().T @ proj) @ (np.kron(b1,np.kron(b2,b3)) @ ghz)
            print(f"Pr({i}{j}{k}) = {round(abs(prob), 2)}")

Pr(000) = 0.25
Pr(001) = 0.0
Pr(010) = 0.0
Pr(011) = 0.25
Pr(100) = 0.25
Pr(101) = 0.0
Pr(110) = 0.0
Pr(111) = 0.25
